#### What does ```enum``` module do?
- It provides an implementation of an *enumeration* type, with iteration and comparison capabilities. 
- It can be used to create well-defined symbols for values, instead of using literal or integers.
- 另附上 [源代码](https://github.com/python/cpython/blob/3.7/Lib/enum.py)

#### 注意
- 此文件是以 [PyMOTW3 - Enum](https://pymotw.com/3/enum/index.html#module-enum) 为学习基础
- 以它为始, 再至标准库巩固

<p></p>

In [31]:
import enum 

class BodyStatus(enum.Enum):
    ''' Defined by subclassing Enum
    '''
    read = 5
    sleep = 4
    work = 3
    fun = 2 
    eat = 1 

In [33]:
# each inst has two props: (name,value)
print('Member name: {}'.format(BodyStatus.work.name))
print('Member value: {}\n'.format(BodyStatus.work.value))

# iterate it
for status in BodyStatus:
    print('{:5} = {}'.format(status.name,status.value))

Member name: work
Member value: 3

read  = 5
sleep = 4
work  = 3
fun   = 2
eat   = 1


In [51]:
# basic comparison 

actual_state = BodyStatus.fun
desired_state = BodyStatus.work 

print('Equality:',
     actual_state == desired_state,  # normal 
     actual_state == BodyStatus.fun) # important

print('Identity:',
     actual_state is desired_state,  # normal 
     actual_state is BodyStatus.fun) # important

Equality: False True
Identity: False True


In [58]:
# cannot sort directly 
try: 
    sorted(BodyStatus)
except TypeError as err:
    print('\nCannot sort: {}'.format(err))


Cannot sort: '<' not supported between instances of 'BodyStatus' and 'BodyStatus'


In [59]:
# able to sort now ( by subclassing "IntEnum" )

class BodyStatusLikeNum(enum.IntEnum):
    # By subclassing 'IntEnum' it behaves more like numbers 
    read = 5
    sleep = 4
    work = 3
    fun = 2 
    eat = 1 
    
try:
    [ i for i in sorted(BodyStatusLikeNum)]
except TypeError as err:
    print('Nah..')

[<BodyStatusLikeNum.eat: 1>,
 <BodyStatusLikeNum.fun: 2>,
 <BodyStatusLikeNum.work: 3>,
 <BodyStatusLikeNum.sleep: 4>,
 <BodyStatusLikeNum.read: 5>]

In [100]:
class UniqueStatus(enum.Enum):
    ''' Members with the same val are tracked as alias refs to the same obj.
        Aliases will NOT shown in the iterator for the Enum. 
        ... 
        first came, nice boi. 
        last came, you two became ONE (same object)
    '''
    yes = 1
    no = 0
    
    ohshit = 0
    
for status in UniqueStatus:
    print('{:5} = {}'.format(status.name,status.value))

print(
    # FIRST name attached to the value is the Boss.
    UniqueStatus.no == UniqueStatus.ohshit,
    
    # actual 'Equal' (same object)
    UniqueStatus.no is UniqueStatus.ohshit,
    id(UniqueStatus.no) == id(UniqueStatus.ohshit),
    
    sep='\n'
)

yes   = 1
no    = 0
True
True
True


In [103]:
try:
   
    @enum.unique
    class ForceUniqueStatus(enum.Enum):
        ''' By adding this decorator => { repeat? --> error! }
        '''
        yes = 1 
        no = 0 

        nope = 0  # you sure? 
        
except ValueError as err:
    print(err)

duplicate values found in <enum 'ForceUniqueStatus'>: nope -> no


In [123]:
# Create by passing data (value/names)
#     [value]  the enum object's name 
#     [names]  the members in it ( vals are automatically assigned ) 

EnumCreateByData = enum.Enum(
    value = 'CreateEnumByData',         # object's name 
    names = ('morning afternoon night') # vals start with 1
)

for item in EnumCreateByData:
    print("{:10}: {}".format(
        item.name,item.value
    ))

morning   : 1
afternoon : 2
night     : 3


In [125]:
# Another implementation (more detailed)

EnumCreateByDataDetail = enum.Enum(
    value = 'EnumCreateByDataDetail',
    names = [
        ('true',1),  # ('...') => [(e,v),(e,v)]
        ('false',0)
    ],
)

for item in EnumCreateByDataDetail:
    print("{:10}: {}".format(
        item.name,item.value
    ))

true      : 1
false     : 0


In [137]:

class EnumTupleValues(enum.Enum):
    ''' Any type of objects can be associated with a member.
            If the value is a tuple, 
            the members are passed as individual args to __init__()
    '''
    start = (4,['one','two','three'])
    one = (1,['one'])
    two = (2,['two'])
    three = (3,['three'])
    
    def __init__(self,num,items):
        self.num = num
        self.items = items 
        
    def in_it(self,new_arg):
        return new_arg.name in self.items 
    
    
EnumTupleValues.start 

EnumTupleValues.start.value 
EnumTupleValues.start.items

# ['one'] in ['one','two','three']
EnumTupleValues.start.in_it(EnumTupleValues.one)

<EnumTupleValues.start: (4, ['one', 'two', 'three'])>

(4, ['one', 'two', 'three'])

['one', 'two', 'three']

True

In [146]:
# Extending its usage (dict)

class EnumComplexValues(enum.Enum):
    ''' For more complex cases, tuples might be unwieldy. (笨重)
        Dict can be used for 
            cases where there are a lot of 
            separate attrs to track for each enum value.
        Complex vals are passed directly 
            to __init__() as the only arg other than self 
    '''
    new = {
        'num': 7,
        'transitions': [
            'incomplete',
            'invalid',
            'wont_fix',
            'in_progress',
        ],
    }
    incomplete = {
        'num': 6,
        'transitions': ['new', 'wont_fix'],
    }
    invalid = {
        'num': 5,
        'transitions': ['new'],
    }
    wont_fix = {
        'num': 4,
        'transitions': ['new'],
    }
    in_progress = {
        'num': 3,
        'transitions': ['new','fix_committed'],
    }
    fix_committed = {
        'num': 2,
        'transitions': ['in_progress','fix_released'],
    }
    fix_released = {
        'num': 1,
        'transitions': ['new'],
    }
    
    def __init__(self,vals):
        self.num = vals['num']
        self.transitions = vals['transitions']
        
    def can_transition(self,new_state):
        return new_state.name in self.transitions
    
    
# continue (UP)

EnumComplexValues.in_progress

EnumComplexValues.in_progress.name 
EnumComplexValues.in_progress.transitions

# ['new', 'fix_committed'] 
#     in ['incomplete', 'invalid', 'wont_fix', 'in_progress']
EnumComplexValues.in_progress.can_transition(EnumComplexValues.new)

<EnumComplexValues.in_progress: {'num': 3, 'transitions': ['new', 'fix_committed']}>

'in_progress'

['new', 'fix_committed']

True